In [1]:
using BenchmarkTools
using Base.Sort

In [2]:
function f!(a, b)
    for x in b
        push!(a, x)
        sort!(a)
    end
    a
end

function g!(a, b)
    append!(a, b)
    sort!(a)
end

function h!(a, b)
    for x in b
        i = Base.Sort.searchsortedfirst(a, x)
        insert!(a, i, x)
    end
    a
end

h! (generic function with 1 method)

In [13]:
a, b = sort(rand(1000)), rand(10)

([0.0020643061514924455, 0.003159625469048244, 0.004698699657677219, 0.004951307470823618, 0.005119495927695894, 0.005686257941762474, 0.005737246710419974, 0.006293463018844259, 0.0065981903672351905, 0.006817309334387733  …  0.9921246611004808, 0.9925083638691115, 0.9925266203001877, 0.9946039414083103, 0.9966202139248339, 0.9966547034276048, 0.9972614083494313, 0.997455688235342, 0.9976310779076223, 0.9977889657854624], [0.15944570583984619, 0.45953224718794794, 0.27647902085294884, 0.933811892138106, 0.2612007835974035, 0.38150261028509624, 0.1845282246789035, 0.09380565053461787, 0.09712827811891844, 0.2817566460678367])

In [4]:
f!(copy(a), b) == g!(copy(a), b) == h!(copy(a), b)

true

In [5]:
@btime f!(A, $b) setup=(A = copy(a));

  48.300 μs (1 allocation: 15.69 KiB)


In [6]:
@btime g!(A, $b) setup=(A = copy(a));

  6.000 μs (1 allocation: 15.69 KiB)


In [7]:
@btime h!(A, $b) setup=(A = copy(a));

  706.250 ns (0 allocations: 1004 bytes)


In [8]:
@btime f!(A, $b) setup=(A = copy(a); sizehint!(A, length(a)+length(b)));

  47.500 μs (0 allocations: 0 bytes)


In [9]:
@btime g!(A, $b) setup=(A = copy(a); sizehint!(A, length(a)+length(b)));

  5.300 μs (0 allocations: 0 bytes)


In [10]:
@btime h!(A, $b) setup=(A = copy(a); sizehint!(A, length(a)+length(b)));

  717.647 ns (0 allocations: 0 bytes)


In [18]:
A = copy(a)
@btime begin
    for i in 1:10
        push!(A, b[i])
        sort!(A)
    end
    A
end

  3.129 ms (10 allocations: 160 bytes)


48200-element Vector{Float64}:
 0.0020643061514924455
 0.003159625469048244
 0.004698699657677219
 0.004951307470823618
 0.005119495927695894
 0.005686257941762474
 0.005737246710419974
 0.006293463018844259
 0.0065981903672351905
 0.006817309334387733
 0.007284505129242502
 0.00832386523540074
 0.011937050227307067
 ⋮
 0.9917959829385872
 0.9918062433272767
 0.9921246611004808
 0.9925083638691115
 0.9925266203001877
 0.9946039414083103
 0.9966202139248339
 0.9966547034276048
 0.9972614083494313
 0.997455688235342
 0.9976310779076223
 0.9977889657854624

In [19]:
@btime begin
    for i in 1:10
        push!(B, b[i])
        sort!(B)
    end
    B
end setup=(B = copy(a); sizehint!(B, length(a)+length(b)));

  53.500 μs (10 allocations: 160 bytes)
